# Setup

## Imports

In [1]:
import Plotly from "tslab-plotly";
import * as tslab from "tslab";
import assert from "assert/strict";

## Shared layout

In [2]:
const sharedLayout = {
    width: 600,
    height: 400,
    font: {
        size: 15,
    },
    margin: {
        l: 80,
        r: 4,
        b: 40,
        t: 50, // for title
    },
};

# Plotly demo

In [3]:
(() => {
    let trace1: Plotly.Data = {
        x: ['giraffes', 'orangutans', 'monkeys'],
        y: [20, 14, 23],
        name: 'Zoo 1',
        type: 'bar',
    };

    let trace2: Plotly.Data = {
        x: ['giraffes', 'orangutans', 'monkeys'],
        y: [12, 18, 29],
        name: 'Zoo 2',
        type: 'bar',
    };

    let trace3: Plotly.Data = {
        x: ['penguins', 'crocodiles', 'swans'],
        y: [16, 10, 28],
        name: 'Zoo 3',
        type: 'bar',

        xaxis: 'x2',
        yaxis: 'y2',
    };

    let trace4: Plotly.Data = {
        x: ['penguins', 'crocodiles', 'swans'],
        y: [24, 12, 16],
        name: 'Zoo 4',
        type: 'bar',

        xaxis: 'x2',
        yaxis: 'y2',
    };

    let data: Plotly.Data[] = [trace1, trace2, trace3, trace4];
    let layout: Partial<Plotly.Layout> = {
        ...sharedLayout,

        grid: { rows: 1, columns: 2, pattern: 'independent' },
        title: 'Animals in different zoos',
        barmode: 'group',
        annotations: [
            // subplot titles (hack)
            {
                text: 'First subplot',
                font: {
                    size: 16,
                },
                align: 'center',
                x: 0.13, //position in x domain
                y: 1.1, //position in y domain
                xref: 'paper',
                yref: 'paper',
                showarrow: false,
            },
            {
                text: 'Second subplot',
                font: {
                    size: 16,
                },
                align: 'center',
                x: 0.9, //position in x domain
                y: 1.1, // position in y domain
                xref: 'paper',
                yref: 'paper',
                showarrow: false,
            },
        ],
    };
    Plotly.newPlot(tslab, data, layout);
})();


# Loading report collections

In [4]:
import fs from 'fs';
import unpackUnityJson from './util/unpackUnityJson';
import loadReportCollection from './util/loadReportCollection';
import path from 'path';

const reportCollections = (() => {
    const reportDir = './Reports/';
    const reportCollections = {};
    for (const reportFileName of fs.readdirSync(reportDir)) {
        const reportPath = path.join(reportDir, reportFileName);

        let reportName = reportFileName;
        const reportNameParts = reportFileName.split('.');
        reportNameParts.splice(-1, 1);
        reportName = reportNameParts.join('');

        reportCollections[reportName] = loadReportCollection(
            unpackUnityJson(JSON.parse(fs.readFileSync(reportPath, 'utf-8')))
        ).reports;
    }

    return reportCollections;
})();

## Report format

In [5]:
(() => {
    const keys = Object.keys(reportCollections);
    console.dir({
        reports: keys,
        launchParameters: reportCollections[keys[0]][0].launchParameters,
    });
})();


{
  reports: [
    'Algorithm convergence',
    'Empty cluster randomization',
    'Random swap (1KM) vs Random swap (2KM)',
    'Scaling vs subsampling',
    'Scanline jitter',
    'Staggered jitter',
    'Subsampling'
  ],
  launchParameters: {
    dispatcherParameters: { stopCondition: true, p: 3 },
    videoName: '2',
    numIterations: 1,
    workingTextureSize: 256,
    numClusters: 6,
    jitterSize: 1,
    staggeredJitter: false,
    doDownscale: false,
    algorithm: 'KHM',
    doRandomizeEmptyClusters: false,
    stopCondition: true
  }
}


# Plots

## Subsampling

In [6]:
(() => {
    const reports = reportCollections['Subsampling'];

    const textureSizeOptions = Array.from(
        new Set(
            reports.map(
                (report) => report.launchParameters.workingTextureSize
            )
        )
    );

    const videoNameOptions = Array.from(
        new Set(
            reports.map(
                (report) => report.launchParameters.videoName
            )
        )
    ).sort();

    for (const aggregatedStatName of ['mean', 'peak']) {
        for (const videoName of videoNameOptions) {
            const subsetVideo = reports.filter(
                (report) => report.launchParameters.videoName == videoName
            );

            const data: Plotly.Data[] = textureSizeOptions.map(
                (textureSize) => {
                    const subsetTextureSize = subsetVideo
                        .filter(
                            (report) =>
                                report.launchParameters.workingTextureSize ==
                                textureSize
                        ).reverse();

                    return {
                        y: subsetTextureSize.map(
                            (report) =>
                                report.measurement.aggregated[
                                    aggregatedStatName
                                ]
                        ),

                        x: subsetTextureSize.map(
                            (report) => `${report.launchParameters.numClusters}`
                        ),

                        name: `${textureSize}`,
                        type: 'bar',
                    };
                }
            );

            // todo fix definitions Plotly.Layout
            const layout: any /*Partial<Plotly.Layout>*/ = {
                ...sharedLayout,

                xaxis: {
                    title: 'number of clusters',
                    type: 'category',
                },

                yaxis: {
                    title: {
                        text: `${aggregatedStatName} variance`,
                        standoff: 4,
                    },
                    automargin: true,
                },

                legend: {
                    title: {
                        text: 'texture size',
                    },
                },

                title: `Subsampling (video ${videoName})`,
            };

            Plotly.newPlot(tslab, data, layout);
            tslab.display.html('<br>');
        }
    }
})();


## Subsample vs Downscale

In [7]:
(() => {
    const reports = reportCollections['Scaling vs subsampling'];

    const doDownscaleOptions = Array.from(
        new Set(
            reports.map(
                (report) => report.launchParameters.doDownscale
            )
        )
    );

    const videoNameOptions = Array.from(
        new Set(
            reports.map(
                (report) => report.launchParameters.videoName
            )
        )
    ).sort();

    for (const aggregatedStatName of ['mean', 'peak']) {
        for (const videoName of videoNameOptions) {
            const subsetVideo = reports.filter(
                (report) => report.launchParameters.videoName == videoName
            );

            const data: Plotly.Data[] = doDownscaleOptions.map(
                (doDownscale) => {
                    const subsetDoDownscale = subsetVideo.filter(
                        (report) =>
                            report.launchParameters.doDownscale == doDownscale
                    );

                    return {
                        y: subsetDoDownscale.map(
                            (report) =>
                                report.measurement.aggregated[
                                    aggregatedStatName
                                ]
                        ),

                        x: subsetDoDownscale.map(
                            (report) =>
                                `${report.launchParameters.workingTextureSize}`
                        ),

                        name: `${doDownscale ? 'downscale' : 'subsample'}`,
                        type: 'bar',
                    };
                }
            );

            // todo fix definitions Plotly.Layout
            const layout: any /*Partial<Plotly.Layout>*/ = {
                ...sharedLayout,

                xaxis: {
                    title: 'working texture size',
                    type: 'category',
                    automargin: true,
                },

                yaxis: {
                    title: {
                        text: `${aggregatedStatName} variance`,
                        standoff: 4,
                    },
                    automargin: true,
                },

                title: `Downscaling vs subsampling (video ${videoName})`,
            };

            Plotly.newPlot(tslab, data, layout);
            tslab.display.html('<br>');
        }
    }
})();


## Jitter

### Scanline jitter

In [8]:
(() => {
    const reports = reportCollections['Scanline jitter'];

    const jitterSizeOptions: number[] = Array.from(
        new Set(
            reports.map((report) =>
                Number.parseInt(report.launchParameters.jitterSize)
            )
        )
    );
    jitterSizeOptions.sort((a, b) => a-b);

    const videoNameOptions = Array.from(
        new Set(
            reports.map(
                (report) => report.launchParameters.videoName
            )
        )
    ).sort();

    for (const aggregatedStatName of ['mean', 'peak']) {
        for (const videoName of videoNameOptions) {
            const subsetVideo = reports.filter(
                (report) => report.launchParameters.videoName == videoName
            );

            const data: Plotly.Data[] = jitterSizeOptions.map(
                (jitterSize) => {
                    const subsetJitterSize = subsetVideo.filter(
                        (report) =>
                            report.launchParameters.jitterSize == jitterSize
                    );

                    return {
                        y: subsetJitterSize.map(
                            (report) =>
                                report.measurement.aggregated[
                                    aggregatedStatName
                                ]
                        ),

                        x: subsetJitterSize.map(
                            (report) =>
                                `${report.launchParameters.workingTextureSize}`
                        ),

                        name: `${jitterSize}`,
                        type: 'bar',
                    };
                }
            );

            // todo fix definitions Plotly.Layout
            const layout: any /*Partial<Plotly.Layout>*/ = {
                ...sharedLayout,

                xaxis: {
                    title: 'working texture size',
                    type: 'category',
                    automargin: true,
                },

                yaxis: {
                    title: {
                        text: `${aggregatedStatName} variance`,
                        standoff: 4,
                    },
                    automargin: true,
                },

                legend: {
                    title: {
                        text: "jitter size"
                    }
                },

                title: `Scanline jitter (video ${videoName})`,
            };

            Plotly.newPlot(tslab, data, layout);
            tslab.display.html('<br>');
        }
    }
})();

### Staggered jitter

In [9]:
(() => {
    const reports = reportCollections['Staggered jitter'];

    const jitterSizeOptions: number[] = Array.from(
        new Set(
            reports.map((report) =>
                Number.parseInt(report.launchParameters.jitterSize)
            )
        )
    );
    jitterSizeOptions.sort((a, b) => a-b);

    const videoNameOptions = Array.from(
        new Set(
            reports.map(
                (report) => report.launchParameters.videoName
            )
        )
    ).sort();

    for (const aggregatedStatName of ['mean', 'peak']) {
        for (const videoName of videoNameOptions) {
            const subsetVideo = reports.filter(
                (report) => report.launchParameters.videoName == videoName
            );

            const data: Plotly.Data[] = jitterSizeOptions.map(
                (jitterSize) => {
                    const subsetJitterSize = subsetVideo.filter(
                        (report) =>
                            report.launchParameters.jitterSize == jitterSize
                    );

                    return {
                        y: subsetJitterSize.map(
                            (report) =>
                                report.measurement.aggregated[
                                    aggregatedStatName
                                ]
                        ),

                        x: subsetJitterSize.map(
                            (report) =>
                                `${report.launchParameters.workingTextureSize}`
                        ),

                        name: `${jitterSize}`,
                        type: 'bar',
                    };
                }
            );

            // todo fix definitions Plotly.Layout
            const layout: any /*Partial<Plotly.Layout>*/ = {
                ...sharedLayout,

                xaxis: {
                    title: 'working texture size',
                    type: 'category',
                    automargin: true,
                },

                yaxis: {
                    title: {
                        text: `${aggregatedStatName} variance`,
                        standoff: 4,
                    },
                    automargin: true,
                },

                legend: {
                    title: {
                        text: "jitter size"
                    }
                },

                title: `Staggered jitter (video ${videoName})`,
            };

            Plotly.newPlot(tslab, data, layout);
            tslab.display.html('<br>');
        }
    }
})();

### Difference

In [10]:
import { json } from 'stream/consumers';

(() => {
    const scanlineJitterReports = reportCollections['Scanline jitter'];
    const staggeredJitterReports = reportCollections['Staggered jitter'];

    const reportPairs = scanlineJitterReports.map((scanlineJitterReport) => {
        const launchParamsString = JSON.stringify(
            scanlineJitterReport.launchParameters
        );

        const staggeredJitterReport = staggeredJitterReports.find(
            (report) =>
                JSON.stringify(report.launchParameters) == launchParamsString
        );

        return {
            scanlineJitterReport: scanlineJitterReport,
            staggeredJitterReport: staggeredJitterReport,
            launchParameters: scanlineJitterReport.launchParameters,
        };
    });

    const jitterSizeOptions: number[] = Array.from(
        new Set(
            scanlineJitterReports.map((report) =>
                Number.parseInt(report.launchParameters.jitterSize)
            )
        )
    );
    jitterSizeOptions.sort((a, b) => a-b);

    const videoNameOptions = Array.from(
        new Set(
            scanlineJitterReports.map(
                (report) => report.launchParameters.videoName
            )
        )
    ).sort();

    for (const aggregatedStatName of ['mean', 'peak']) {
        for (const videoName of videoNameOptions) {
            const subsetVideo = reportPairs.filter(
                (reportPair) =>
                    reportPair.launchParameters.videoName == videoName
            );

            const data: Plotly.Data[] = jitterSizeOptions.map((jitterSize) => {
                const subsetJitterSize = subsetVideo.filter(
                    (reportPair) =>
                        reportPair.launchParameters.jitterSize == jitterSize
                );

                return {
                    y: subsetJitterSize.map(
                        (reportPair) =>
                            reportPair.scanlineJitterReport.measurement
                                .aggregated[aggregatedStatName] -
                            reportPair.staggeredJitterReport.measurement
                                .aggregated[aggregatedStatName]
                    ),

                    x: subsetJitterSize.map(
                        (reportPair) =>
                            `${reportPair.launchParameters.workingTextureSize}`
                    ),

                    name: `${jitterSize}`,
                    type: 'bar',
                };
            });

            // todo fix definitions Plotly.Layout
            const layout: any /*Partial<Plotly.Layout>*/ = {
                ...sharedLayout,

                xaxis: {
                    title: 'working texture size',
                    type: 'category',
                    automargin: true,
                },

                yaxis: {
                    title: {
                        text: `Δ ${aggregatedStatName} variance`,
                        standoff: 4,
                    },
                    automargin: true,
                },

                legend: {
                    title: {
                        text: 'jitter size',
                    },
                },

                title: `Scanline jitter minus staggered jitter (video ${videoName})`,
            };

            Plotly.newPlot(tslab, data, layout);
            tslab.display.html('<br>');
        }
    }
})();


## Empty cluster randomization

In [11]:
(() => {
    const reports = reportCollections['Empty cluster randomization'];

    const videoNameOptions = Array.from(
        new Set(
            reports.map(
                (report) => report.launchParameters.videoName
            )
        )
    ).sort();

    for (const aggregatedStatName of ['mean', 'peak']) {
        for (const videoName of videoNameOptions) {
            const subsetVideo = reports.filter(
                (report) => report.launchParameters.videoName == videoName
            );

            const data: Plotly.Data[] = [true, false].map(
                (doRandomizeEmptyClusters) => {
                    const subsetJitterSize = subsetVideo.filter(
                        (report) =>
                            report.launchParameters.doRandomizeEmptyClusters == doRandomizeEmptyClusters
                    );

                    return {
                        y: subsetJitterSize.map(
                            (report) =>
                                report.measurement.aggregated[
                                    aggregatedStatName
                                ]
                        ),

                        x: subsetJitterSize.map(
                            (report) =>
                                `${report.launchParameters.workingTextureSize}`
                        ),

                        name: `${doRandomizeEmptyClusters ? "randomized" : "unchanged"}`,
                        type: 'bar',
                    };
                }
            );

            // todo fix definitions Plotly.Layout
            const layout: any /*Partial<Plotly.Layout>*/ = {
                ...sharedLayout,

                xaxis: {
                    title: 'working texture size',
                    type: 'category',
                    automargin: true,
                },

                yaxis: {
                    title: {
                        text: `${aggregatedStatName} variance`,
                        standoff: 4,
                    },
                    automargin: true,
                },

                legend: {
                    title: {
                        text: "empty clusters"
                    }
                },

                margin: {
                    l: 80,
                    r: 4,
                    b: 40,
                    t: 50, // for title
                },

                title: `Empty cluster randomization (video ${videoName})`,
            };

            Plotly.newPlot(tslab, data, layout);
            tslab.display.html('<br>');
        }
    }
})();

## Random Swap - KM iterations

In [12]:
(() => {
    const reports = reportCollections['Random swap (1KM) vs Random swap (2KM)'];

    const numIterationsKmOptions: number[] = Array.from(
        new Set(
            reports.map((report) =>
                Number.parseInt(
                    report.launchParameters.dispatcherParameters.numIterationsKm
                )
            )
        )
    );
    numIterationsKmOptions.sort((a, b) => a - b);

    const videoNameOptions = Array.from(
        new Set(reports.map((report) => report.launchParameters.videoName))
    ).sort();

    for (const aggregatedStatName of ['mean', 'peak']) {
        for (const videoName of videoNameOptions) {
            const subsetVideo = reports.filter(
                (report) => report.launchParameters.videoName == videoName
            );

            const data: Plotly.Data[] = numIterationsKmOptions.map(
                (numIterationsKm) => {
                    const subsetNumIterationsKm = subsetVideo
                        .filter(
                            (report) =>
                                report.launchParameters.dispatcherParameters
                                    .numIterationsKm == numIterationsKm
                        )
                        .sort(
                            (reportA, reportB) =>
                                reportA.launchParameters.numIterations -
                                reportB.launchParameters.numIterations
                        );

                    return {
                        y: subsetNumIterationsKm.map(
                            (report) =>
                                report.measurement.aggregated[
                                    aggregatedStatName
                                ]
                        ),

                        x: subsetNumIterationsKm.map(
                            (report) =>
                                `${report.launchParameters.numIterations}`
                        ),

                        name: `${numIterationsKm}`,
                        type: 'scatter',
                        mode: 'lines'
                    };
                }
            );

            // todo fix definitions Plotly.Layout
            const layout: any /*Partial<Plotly.Layout>*/ = {
                ...sharedLayout,

                xaxis: {
                    title: 'total KM iterations',
                    type: 'category',
                    automargin: true,
                },

                yaxis: {
                    title: {
                        text: `${aggregatedStatName} variance`,
                        standoff: 4,
                    },
                    automargin: true,
                },

                legend: {
                    title: {
                        text: 'KM iterations per swap',
                    },
                },

                title: `Scanline jitter (video ${videoName})`,
            };

            Plotly.newPlot(tslab, data, layout);
            tslab.display.html('<br>');
        }
    }
})();


## Algorithm covnergence

In [24]:
(() => {
    const reports = reportCollections['Algorithm convergence'];

    const workingTextureSizeOptions = Array.from(
        new Set(
            reports.map((report) =>
                Number.parseInt(report.launchParameters.workingTextureSize)
            )
        )
    );
    workingTextureSizeOptions.sort();

    const algorithmOptions = Array.from(
        new Set(reports.map((report) => report.launchParameters.algorithm))
    );
    algorithmOptions.sort();
    console.dir(algorithmOptions);

    const videoNameOptions = Array.from(
        new Set(reports.map((report) => report.launchParameters.videoName))
    ).sort();

    for (const workingTextureSize of workingTextureSizeOptions) {
        tslab.display.markdown(
            `### Working texture size: ${workingTextureSize}`
        );

        const subsetTextureSize = reports.filter(
            (report) =>
                report.launchParameters.workingTextureSize == workingTextureSize
        );

        for (const aggregatedStatName of ['mean', 'peak']) {
            for (const videoName of videoNameOptions) {
                const subsetVideo = subsetTextureSize.filter(
                    (report) => report.launchParameters.videoName == videoName
                );

                let data: Plotly.Data[] = [];

                for (const stopCondition of [true, false]) {
                    data = data.concat(
                        algorithmOptions.map((algorithm) => {
                            let subsetAlgorithm = subsetVideo
                                .filter(
                                    (report) =>
                                        report.launchParameters.algorithm ==
                                        algorithm
                                )
                                .sort(
                                    (reportA, reportB) =>
                                        reportA.launchParameters.numIterations -
                                        reportB.launchParameters.numIterations
                                )
                                .filter(
                                    (report) =>
                                        report.launchParameters
                                            .dispatcherParameters
                                            .stopCondition == stopCondition
                                );

                            if (subsetAlgorithm.length == 0) {
                                return {};
                            }

                            let x = null,
                                y = null;
                            if (stopCondition) {
                                const maxNumIterations = Math.max(
                                    ...reports.map(
                                        (report) =>
                                            report.launchParameters
                                                .numIterations
                                    )
                                );
                                const minNumIterations = Math.min(
                                    ...reports.map(
                                        (report) =>
                                            report.launchParameters
                                                .numIterations
                                    )
                                );
                                const numIterationsOptions = [
                                    ...new Array(
                                        maxNumIterations - minNumIterations + 1
                                    ).keys(),
                                ];

                                const report = subsetAlgorithm[0];
                                assert(
                                    report.launchParameters.dispatcherParameters
                                        .stopCondition
                                );
                                x = numIterationsOptions;
                                y = numIterationsOptions.map(
                                    () =>
                                        report.measurement.aggregated[
                                            aggregatedStatName
                                        ]
                                );
                            } else {
                                x = subsetAlgorithm.map(
                                    (report) =>
                                        `${report.launchParameters.numIterations}`
                                );

                                y = subsetAlgorithm.map(
                                    (report) =>
                                        report.measurement.aggregated[
                                            aggregatedStatName
                                        ]
                                );
                            }

                            return {
                                y: y,

                                x: x,

                                name: `${algorithm}${
                                    stopCondition ? '(stop condition)' : ''
                                }`,
                                type: 'scatter',
                                mode: 'lines',
                                line: {
                                    width: 5,
                                },

                                legendgroup: `stopCondition: ${stopCondition}`,
                                legendgrouptitle : {
                                    text: `${stopCondition} ? 'stop condition' : 'fixed iterations'`,
                                },
                            };
                        })
                    );
                }

                // todo fix definitions Plotly.Layout
                const layout: any /*Partial<Plotly.Layout>*/ = {
                    ...sharedLayout,

                    xaxis: {
                        title: 'total KM iterations',
                        type: 'category',
                        automargin: true,
                    },

                    yaxis: {
                        title: {
                            text: `${aggregatedStatName} variance`,
                            standoff: 4,
                        },
                        automargin: true,
                    },

                    legend: {
                        title: {
                            text: 'KM iterations per swap',
                        },
                    },

                    margin: {
                        l: 80,
                        r: 4,
                        b: 40,
                        t: 50, // for title
                    },

                    title: `video ${videoName}; working texture size: ${workingTextureSize}`,
                };

                Plotly.newPlot(tslab, data, layout);
            }
        }
    }
})();


[ 'KHM', 'KM', 'RS' ]


### Working texture size: 256

### Working texture size: 64

### Explanation theory

When the input changes significantly between two frames, getting a better clustering structure in one frame can result in a worse starting position for the subsequent frames.

In [14]:
import { report } from 'process';

(() => {
    const reports = reportCollections['Algorithm convergence'];

    const workingTextureSizeOptions = Array.from(
        new Set(
            reports.map((report) =>
                Number.parseInt(report.launchParameters.workingTextureSize)
            )
        )
    );
    workingTextureSizeOptions.sort();

    const videoNameOptions = Array.from(
        new Set(reports.map((report) => report.launchParameters.videoName))
    ).sort();

    function plotDelta({ algorithm }: { algorithm: string }) {
        for (const workingTextureSize of workingTextureSizeOptions) {
            tslab.display.markdown(
                `#### Working texture size: ${workingTextureSize}`
            );
            for (const videoName of videoNameOptions) {
                const subset = reports.filter(
                    (report) =>
                        report.launchParameters.videoName == videoName &&
                        report.launchParameters.algorithm == algorithm &&
                        report.launchParameters.workingTextureSize ==
                            workingTextureSize
                );

                const report3 = subset.find(
                    (report) => report.launchParameters.numIterations == 3
                );

                const report5 = subset.find(
                    (report) => report.launchParameters.numIterations == 5
                );

                const data: Plotly.Data[] = [
                    {
                        y: Object.entries(
                            report5.measurement.varianceByFrame.map(
                                (entry) => entry.variance
                            )
                        ).map(
                            (entry) =>
                                (entry[1] as number) -
                                report3.measurement.varianceByFrame[entry[0]]
                                    .variance
                        ),

                        x: report5.measurement.varianceByFrame.map(
                            (varianceRecord) => varianceRecord.frameIdex
                        ),
                        name: 'plot',
                        type: 'scatter',
                        mode: 'lines',
                    },
                ];

                // todo fix definitions Plotly.Layout
                const layout: any /*Partial<Plotly.Layout>*/ = {
                    ...sharedLayout,

                    xaxis: {
                        title: 'frame index',
                        type: 'category',
                        automargin: true,
                    },

                    yaxis: {
                        title: {
                            text: `Δ variance`,
                            standoff: 4,
                        },
                        automargin: true,
                    },

                    margin: {
                        l: 80,
                        r: 4,
                        b: 80,
                        t: 80, // for title
                    },

                    title: {
                        text: `${algorithm} variance change from 3 to 5 iterations
<br>
(video ${videoName}; working texture size: ${workingTextureSize})`,
                    },
                };

                Plotly.newPlot(tslab, data, layout);
            }
        }
    }

    plotDelta({ algorithm: 'KHM' });
    tslab.display.markdown(
        `KHM variance barely ever increases with increases number of iterations, because KHM is less dependent on initialization. Getting a better clustering structure on the current frame can not significantly worsen the performance on subsequent frames.`
    );
    plotDelta({ algorithm: 'KM' });
    tslab.display.markdown(
        `Every spike is preceeded by a section of improved variance. When zoomed in, this effect can be observed for smaller spikes as well.

KM is heabily dependent on initialization, so getting a better clustering structure in the current frame can lead to worse initialization subsequent frames.

Though as an overall trend higher number of iterations does lead to smaller variance, this effect is strong enough to create situations, where a small increase to the number of iterations leads to worse overall variance over the duration of the video.`
    );
    plotDelta({ algorithm: 'RS' });
    tslab.display.markdown(
        `RS is less prone to getting stuck in a local optimum, than both KM and KHM, but how fast it converges still depends on initialization. Getting a better clustering structure in the current frame can lead to worse convergence in the next frame.

With a fixed number of iterations this leads to higher variance. With a stopping condition it would lead to longer processing time.
`
    );
})();


#### Working texture size: 256

#### Working texture size: 64

KHM variance barely ever increases with increases number of iterations, because KHM is less dependent on initialization. Getting a better clustering structure on the current frame can not significantly worsen the performance on subsequent frames.

#### Working texture size: 256

#### Working texture size: 64

Every spike is preceeded by a section of improved variance. When zoomed in, this effect can be observed for smaller spikes as well.

KM is heabily dependent on initialization, so getting a better clustering structure in the current frame can lead to worse initialization subsequent frames.

Though as an overall trend higher number of iterations does lead to smaller variance, this effect is strong enough to create situations, where a small increase to the number of iterations leads to worse overall variance over the duration of the video.

#### Working texture size: 256

#### Working texture size: 64

RS is less prone to getting stuck in a local optimum, than both KM and KHM, but how fast it converges still depends on initialization. Getting a better clustering structure in the current frame can lead to worse convergence in the next frame.

With a fixed number of iterations this leads to higher variance. With a stopping condition it would lead to longer processing time.


### Conclusion

It appears Random Swap has difficulty adjusting to a dynamic video, where the possibility of exploiting temporal coherency to get a better initialization becomes limited.

KHM does not benefit as much from temporal coherency, but doesn't suffer as much from its lack.

|Video type|Recommended algorithm|
|---|---|
|Mostly static video|RS|
|Highly dynamic video|KHM|